In [1]:
import sympy as sp
import numpy as np
import time 

N = 3
g = sp.Symbol("g", real = True)

## Creating X coordinates (using sp.Matrix for the 4th stack)

In [2]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,4):
    for j in np.arange(1,10):
        X[str(i)+str(j)] = sp.Symbol("X"+str(i)+str(j), real = True)
            
for i in np.arange(1,10):
    #initialise a matrix of correct shape
    X_init = []
    for j in np.arange(1,N+1):
        row = []
        for k in np.arange(1,N+1):
            row.append(sp.Symbol("a_{"+str(j)+str(k)+"}"))
        X_init.append(row)
    X["4"+str(i)] = sp.Matrix(X_init)
    
    for j in np.arange(N):
        for k in np.arange(N):
            if j == k:
                X["4"+str(i)][j,k] = sp.Symbol("X4"+str(i)+"("+str(j)+","+str(k)+")",real = True)
            elif j > k:
                continue
            else:
                x_ij = sp.Symbol("X4"+str(i)+"("+str(j)+","+str(k)+").real()",real = True)
                y_ij = sp.Symbol("X4"+str(i)+"("+str(j)+","+str(k)+").imag()",real = True)
                X["4"+str(i)][j,k] = x_ij + sp.I*y_ij
                X["4"+str(i)][k,j] = x_ij - sp.I*y_ij

## V coordinates (X_dot)

In [10]:
# Creating a dictionary to store the V (X_dot) coordinates
V = {}
for i in np.arange(1,4):
    for j in np.arange(1,10):
        V[str(i)+str(j)] = sp.Symbol("V"+str(i)+str(j), real = True)
            
for i in np.arange(1,10):
    #initialise a matrix of correct shape
    V_init = []
    for j in np.arange(1,N+1):
        row = []
        for k in np.arange(1,N+1):
            row.append(sp.Symbol("a_{"+str(j)+str(k)+"}"))
        V_init.append(row)
    V["4"+str(i)] = sp.Matrix(V_init)
    
    for j in np.arange(N):
        for k in np.arange(N):
            if j == k:
                V["4"+str(i)][j,k] = sp.Symbol("V4"+str(i)+"("+str(j)+","+str(k)+")",real = True)
            elif j > k:
                continue
            else:
                x_ij = sp.Symbol("V4"+str(i)+"("+str(j)+","+str(k)+").real()",real = True)
                y_ij = sp.Symbol("V4"+str(i)+"("+str(j)+","+str(k)+").imag()",real = True)
                V["4"+str(i)][j,k] = x_ij + sp.I*y_ij
                V["4"+str(i)][k,j] = x_ij - sp.I*y_ij

## Creating X coordinates (using sp.MatrixSymbol for the 4th stack)

In [4]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,5):
    for j in np.arange(1,10):
        if i != 4:
            X[str(i)+str(j)] = sp.Symbol("X^{"+str(i)+"}_"+str(j), real = True)
        else:
            X[str(i)+str(j)] = sp.MatrixSymbol("X^{"+str(i)+"}_"+str(j),N,N)

## Creating the $\phi$'s and $\phi^{\dagger}$'s and $\phi^{(ij)}$

In [19]:
phi = {}
phi_ = {}
phi_dag = {}
for i in np.arange(1,5):
    phi[str(i)+"1"] = 1/sp.sqrt(2) * (X[str(i)+"4"] + sp.I * X[str(i)+"5"])
    phi[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] + sp.I * X[str(i)+"7"])
    phi[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] + sp.I * X[str(i)+"9"])
    
    phi_dag[str(i)+"1"] = 1/sp.sqrt(2) * ( X[str(i)+"4"] - sp.I * X[str(i)+"5"])
    phi_dag[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] - sp.I * X[str(i)+"7"])
    phi_dag[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] - sp.I * X[str(i)+"9"])
    
phi_["12"] = phi["13"] - phi["23"]
phi_["23"] = phi["21"] - phi["31"]
phi_["31"] = phi["32"] - phi["12"]

## Creating Z coordinates

In [20]:
# Building the Z-dictionary
Z = {}
for i in np.arange(1,4):
    #Scalars here
    for j in np.arange(1,4):
        if i == j: 
            continue
        else:
            P = sp.Symbol("P"+str(i) + str(j), real = True)
            Q = sp.Symbol("Q"+str(i) + str(j), real = True)
            Z[str(i) + str(j)] = (P + sp.I * Q)
    
    #Vectors here
    store_current_4i = []
    store_current_i4 = []
    for j in np.arange(N):
        P4i = sp.Symbol("P"+str(4)+str(i)+"("+str(j)+")", real = True)
        Q4i = sp.Symbol("Q"+str(4)+str(i)+"("+str(j)+")", real = True)
        Pi4 = sp.Symbol("P"+str(i)+str(4)+"("+str(j)+")", real = True)
        Qi4 = sp.Symbol("Q"+str(i)+str(4)+"("+str(j)+")", real = True)
        store_current_4i.append([P4i + sp.I * Q4i])
        store_current_i4.append([Pi4 + sp.I * Qi4])
    Z["4"+str(i)] = sp.Matrix(store_current_4i)
    Z[str(i) + "4"] = sp.Matrix(store_current_i4).transpose()

## Z_dots (U + iW)

In [25]:
# Call P_dot and Q_dot U and W respectively
Z_dot = {}
for i in np.arange(1,4):
    #Scalars here
    for j in np.arange(1,4):
        if i == j: 
            continue
        else:
            U = sp.Symbol("U"+str(i) + str(j), real = True)
            W = sp.Symbol("W"+str(i) + str(j), real = True)
            Z_dot[str(i) + str(j)] = (P + sp.I * Q)
    
    #Vectors here
    store_current_4i = []
    store_current_i4 = []
    for j in np.arange(N):
        U4i = sp.Symbol("U"+str(4)+str(i)+"("+str(j)+")", real = True)
        W4i = sp.Symbol("W"+str(4)+str(i)+"("+str(j)+")", real = True)
        Ui4 = sp.Symbol("U"+str(i)+str(4)+"("+str(j)+")", real = True)
        Wi4 = sp.Symbol("W"+str(i)+str(4)+"("+str(j)+")", real = True)
        store_current_4i.append([U4i + sp.I * W4i])
        store_current_i4.append([Ui4 + sp.I * Wi4])
    Z_dot["4"+str(i)] = sp.Matrix(store_current_4i)
    Z_dot[str(i) + "4"] = sp.Matrix(store_current_i4).transpose()

## Adding Kinetic

In [37]:
print(sp.Abs(X["31"]))

Abs(X31)


## Creating the F variables

In [6]:
F = {}

#Scalars
for i in np.arange(1,4):
    for j in np.arange(1,5):
        if i == j:
            continue
        if j == 4:
            F[str(i)+str(j)] = (Z[str(i)+str(j)] * Z[str(j)+str(i)])[0] + N*sp.Symbol("c"+str(i)+str(j))/g**2
        else:
            F[str(i)+str(j)] = Z[str(i)+str(j)] * Z[str(j)+str(i)] + sp.Symbol("c"+str(i)+str(j))/g**2 

F["41"] = Z["41"]*Z["14"] + (sp.Symbol("c41")/g**2)*sp.eye(N) +  phi["42"]*phi["43"] - phi["43"]*phi["42"]
F["42"] = Z["42"]*Z["24"] + (sp.Symbol("c42")/g**2)*sp.eye(N) +  phi["43"]*phi["41"] - phi["41"]*phi["43"]
F["43"] = Z["43"]*Z["34"] + (sp.Symbol("c43")/g**2)*sp.eye(N) -  phi["42"]*phi["41"] + phi["41"]*phi["42"]

## Writing in the G's

In [7]:
G = {}

G["21"] = phi_["12"]*Z["21"] + Z["23"]*Z["31"] + (Z["24"]*Z["41"])[0]
G["12"] = phi_["12"]*Z["12"] + Z["13"]*Z["32"] + (Z["14"]*Z["42"])[0]

G["31"] = phi_["31"]*Z["31"] + Z["32"]*Z["21"] - (Z["34"]*Z["41"])[0]
G["13"] = phi_["31"]*Z["13"] + Z["12"]*Z["23"] + (Z["14"]*Z["43"])[0]

G["32"] = phi_["23"]*Z["32"] + Z["31"]*Z["12"] + (Z["34"]*Z["42"])[0]
G["23"] = phi_["23"]*Z["23"] + Z["21"]*Z["13"] + (Z["24"]*Z["43"])[0]

G["41"] = -phi["41"]*Z["41"] + Z["42"]*Z["21"] + Z["43"]*Z["31"]
G["14"] = -Z["14"]*phi["41"] + Z["12"]*Z["24"] - Z["13"]*Z["34"]

G["42"] = -phi["42"]*Z["42"] + Z["43"]*Z["32"] + Z["41"]*Z["12"]
G["24"] = -Z["24"]*phi["42"] + Z["21"]*Z["14"] + Z["23"]*Z["34"]

G["43"] = -phi["43"]*Z["43"] - Z["41"]*Z["13"] + Z["42"]*Z["23"]
G["34"] = -Z["34"]*phi["43"] + Z["31"]*Z["14"] + Z["32"]*Z["24"]

scalar_G = [G["12"],G["21"],G["13"],G["31"],G["23"],G["32"]]

## Creating $V_F$

In [8]:
start = time.time()
Fsum1 = sp.Symbol("",real = True)
Fsum2 = sp.Symbol("",real = True)
Fsum3 = sp.Symbol("",real = True)
Gsum1 = sp.Symbol("",real = True)
Gsum2 = sp.Symbol("",real = True)
Gsum3 = sp.Symbol("",real = True)

#Term 1
for i in np.arange(1,4):
    for j in np.arange(1,4):
        if i == j:
            continue
        else:
            Fsum1+= sp.re(F[str(i)+str(j)])**2 + sp.im(F[str(i)+str(j)])**2
#Term 2
for i in np.arange(1,4):
    Fsum2 += sp.re(F[str(i)+"4"])**2 + sp.im(F[str(i)+"4"])**2
    
#Term 3
for i in np.arange(1,4):
    for j in np.arange(N):
        for k in np.arange(N):
            Fsum3 += sp.re(F["4"+str(i)][j,k])**2 + sp.im(F["4"+str(i)][j,k])**2
            
#Term 4
for i in np.arange(6):
    Gsum1 += (sp.re(scalar_G[i]))**2 + (sp.im(scalar_G[i]))**2
    
#Term 5
for i in np.arange(1,4):
    for j in np.arange(N):
        Gsum2 += (sp.re(G[str(i)+"4"][j]))**2 + (sp.im(G[str(i)+"4"][j]))**2

#Term 6
for i in np.arange(1,4):
    for j in np.arange(N):
        Gsum3 += (sp.re(G[str(i)+"4"][j]))**2 + (sp.im(G[str(i)+"4"][j]))**2
        
V_F = Fsum1 + Fsum2 + Fsum3 + Gsum1 + Gsum2 + Gsum3

print(time.time() - start)

2.6112191677093506


In [27]:
print(V_F.diff(sp.re(Z["14"]))[0])

-4*P13*(P12*P24(2) - P13*P34(2) - Q12*Q24(2) + Q13*Q34(2) + sqrt(2)*(-P14(2)*X44(2,2) + Q14(2)*X45(2,2))/2 + sqrt(2)*(-P14(0)*X44(0,2).real() + P14(0)*X45(0,2).imag() + Q14(0)*X44(0,2).imag() + Q14(0)*X45(0,2).real())/2 + sqrt(2)*(-P14(1)*X44(1,2).real() + P14(1)*X45(1,2).imag() + Q14(1)*X44(1,2).imag() + Q14(1)*X45(1,2).real())/2) + 4*P23*(P14(2)*P21 + P23*P34(2) - Q14(2)*Q21 - Q23*Q34(2) + sqrt(2)*(-P24(2)*X46(2,2) + Q24(2)*X47(2,2))/2 + sqrt(2)*(-P24(0)*X46(0,2).real() + P24(0)*X47(0,2).imag() + Q24(0)*X46(0,2).imag() + Q24(0)*X47(0,2).real())/2 + sqrt(2)*(-P24(1)*X46(1,2).real() + P24(1)*X47(1,2).imag() + Q24(1)*X46(1,2).imag() + Q24(1)*X47(1,2).real())/2) - 2*P41(2)*(P21*P32 - sqrt(2)*P31*X16/2 + sqrt(2)*P31*X36/2 - P34(0)*P41(0) - P34(1)*P41(1) - P34(2)*P41(2) - Q21*Q32 + sqrt(2)*Q31*X17/2 - sqrt(2)*Q31*X37/2 + Q34(0)*Q41(0) + Q34(1)*Q41(1) + Q34(2)*Q41(2)) + 2*P42(2)*(P12*P31 + sqrt(2)*P32*X24/2 - sqrt(2)*P32*X34/2 + P34(0)*P42(0) + P34(1)*P42(1) + P34(2)*P42(2) - Q12*Q31 - sqrt